In [120]:
#RESEARCH QUESTION :

#1.Identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018.





In [26]:
#IMPORTING THE RELEVANT LIBRARIES 

import numpy as np
import pandas as pd
import plotly as py
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import csv

**LOADING THE DATASETS**

In [27]:
url='/content/Autolib_dataset (2).csv'
df_autolib=pd.read_csv(url)
df_autolib

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,0,operational,Le Bourget,NaN,lebourget-johnfitzgeraldkennedy-2,STATION,"48.938103, 2.4286035",93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,1,operational,Paris,NaN,paris-eugeneoudine-51,STATION,"48.8250327, 2.3725162",75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,8 avenue MÃ©nelotte,2,2,0,0,0,nonexistent,Colombes,NaN,colombes-menelotte-8,STATION,"48.9246525, 2.259313",92700,Colombes/MÃ©nelotte/8,operational,NaN,3,station,ok,nonexistent,2018,4,6,11,26
4996,37 rue de Dantzig,4,4,0,0,1,operational,Paris,NaN,paris-dantzig-37,STATION,"48.8335103, 2.2987201",75015,Paris/Dantzig/37,operational,NaN,2,station,ok,nonexistent,2018,4,4,16,56
4997,142 rue du Bac,1,1,0,0,1,operational,Paris,NaN,paris-bac-142,STATION,"48.8508194, 2.3237968",75007,Paris/Bac/142,operational,NaN,4,station,ok,nonexistent,2018,4,1,7,1
4998,2 avenue du Val de Fontenay,2,2,0,0,0,nonexistent,Fontenay-Sous-Bois,NaN,fontenaysousbois-valdefontenay-2,STATION,"48.8528247, 2.4869085",94120,Fontenay-Sous-Bois/Val de Fontenay/2,operational,NaN,3,station,ok,nonexistent,2018,4,4,17,27


**Exploring the Dataset**

In [28]:
#viewing the summary of the dataframe
df_autolib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Cars                 5000 non-null   int64 
 2   Bluecar counter      5000 non-null   int64 
 3   Utilib counter       5000 non-null   int64 
 4   Utilib 1.4 counter   5000 non-null   int64 
 5   Charge Slots         5000 non-null   int64 
 6   Charging Status      5000 non-null   object
 7   City                 5000 non-null   object
 8   Displayed comment    111 non-null    object
 9   ID                   5000 non-null   object
 10  Kind                 5000 non-null   object
 11  Geo point            5000 non-null   object
 12  Postal code          5000 non-null   int64 
 13  Public name          5000 non-null   object
 14  Rental status        5000 non-null   object
 15  Scheduled at         47 non-null     object
 16  Slots 

Selecting a sample

In [29]:
#to preview the first 5
df_autolib.head()

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,0,operational,Le Bourget,NaN,lebourget-johnfitzgeraldkennedy-2,STATION,"48.938103, 2.4286035",93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,1,operational,Paris,NaN,paris-eugeneoudine-51,STATION,"48.8250327, 2.3725162",75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23


In [30]:
df_autolib.tail()

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
4995,8 avenue MÃ©nelotte,2,2,0,0,0,nonexistent,Colombes,NaN,colombes-menelotte-8,STATION,"48.9246525, 2.259313",92700,Colombes/MÃ©nelotte/8,operational,NaN,3,station,ok,nonexistent,2018,4,6,11,26
4996,37 rue de Dantzig,4,4,0,0,1,operational,Paris,NaN,paris-dantzig-37,STATION,"48.8335103, 2.2987201",75015,Paris/Dantzig/37,operational,NaN,2,station,ok,nonexistent,2018,4,4,16,56
4997,142 rue du Bac,1,1,0,0,1,operational,Paris,NaN,paris-bac-142,STATION,"48.8508194, 2.3237968",75007,Paris/Bac/142,operational,NaN,4,station,ok,nonexistent,2018,4,1,7,1
4998,2 avenue du Val de Fontenay,2,2,0,0,0,nonexistent,Fontenay-Sous-Bois,NaN,fontenaysousbois-valdefontenay-2,STATION,"48.8528247, 2.4869085",94120,Fontenay-Sous-Bois/Val de Fontenay/2,operational,NaN,3,station,ok,nonexistent,2018,4,4,17,27
4999,5 Rue Henry Brisson,4,4,0,0,0,operational,Paris,NaN,paris-henrybrisson-5,STATION,"48.8984293, 2.3347923",75018,Paris/Henry Brisson/5,operational,NaN,0,station,ok,nonexistent,2018,4,8,10,2


In [33]:
sample=df_autolib.sample(10)
sample

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
4996,37 rue de Dantzig,4,4,0,0,1,operational,Paris,NaN,paris-dantzig-37,STATION,"48.8335103, 2.2987201",75015,Paris/Dantzig/37,operational,NaN,2,station,ok,nonexistent,2018,4,4,16,56
4598,35 Rue de Provence,4,4,0,0,0,nonexistent,Paris,NaN,paris-provence-35,STATION,"48.874155, 2.338559",75009,Paris/Provence/35,operational,NaN,0,station,ok,nonexistent,2018,4,4,16,28
4372,10 rue Anatole de la Forge,0,0,0,0,0,nonexistent,Paris,NaN,paris-anatoledelaforge-10,STATION,"48.8758331, 2.292289",75017,Paris/Anatole de la Forge/10,future,2015-09-29T13:58:29,0,station,scheduled,nonexistent,2018,4,9,12,1
1535,2 avenue Georges Pompidou,0,0,0,0,0,nonexistent,Sucy-en-Brie,NaN,sucyenbrie-georgespompidou-2,STATION,"48.7708852, 2.5227425",94370,Sucy-en-Brie/Georges Pompidou/2,future,2016-04-26T14:28:01,0,station,scheduled,nonexistent,2018,4,9,22,58
427,135 avenue de Verdun,5,5,0,0,0,nonexistent,Issy-les-Moulineaux,NaN,issylesmoulineaux-verdun-135,STATION,"48.81855, 2.25291",92130,Issy-les-Moulineaux/Verdun/135,operational,NaN,0,station,ok,nonexistent,2018,4,6,17,50
1735,137 avenue Jean JaurÃ¨s,2,2,0,0,0,nonexistent,Clamart,NaN,clamart-jeanjaures-137,STATION,"48.8080853, 2.2679023",92140,Clamart/Jean JaurÃ¨s/137,operational,NaN,1,station,ok,nonexistent,2018,4,3,12,20
2080,4 Avenue SecrÃ©tan,1,1,0,0,2,operational,Paris,NaN,paris-secretan-4,STATION,"48.8821756, 2.3708912",75019,Paris/SecrÃ©tan/4,operational,NaN,5,station,ok,nonexistent,2018,4,4,19,36
3167,1 Place Georges Pompidou,4,4,0,0,0,nonexistent,Levallois-Perret,"Station en parking (niv -1), accÃ¨s Place Geor...",levalloisperret-pompidou-parking,PARKING,"48.89851, 2.28102",92300,Levallois-Perret/Pompidou/Parking,operational,NaN,0,station,ok,nonexistent,2018,4,3,4,23
2425,97 Rue Georges Guynemer,5,5,1,0,0,nonexistent,AsniÃ¨res-sur-Seine,NaN,asnieressurseine-georgesguynemer-97,SPACE,"48.91587, 2.291084",92600,AsniÃ¨res-sur-Seine/Georges Guynemer/97,operational,NaN,0,full_station,ok,operational,2018,4,4,19,35
2921,24 rue Caillaux,1,1,0,0,1,operational,Paris,NaN,paris-caillaux-24,STATION,"48.82223, 2.358905",75013,Paris/Caillaux/24,operational,NaN,2,station,ok,nonexistent,2018,4,4,0,19


**DATA CLEANING**

In [34]:
df_autolib.columns

Index(['Address', 'Cars', 'Bluecar counter', 'Utilib counter',
       'Utilib 1.4 counter', 'Charge Slots', 'Charging Status', 'City',
       'Displayed comment', 'ID', 'Kind', 'Geo point', 'Postal code',
       'Public name', 'Rental status', 'Scheduled at', 'Slots', 'Station type',
       'Status', 'Subscription status', 'year', 'month', 'day', 'hour',
       'minute'],
      dtype='object')

In [36]:
#dropping the columns that we wont need for the analysis
df_autolib.drop(['Displayed comment','Cars','Geo point','Address'], axis=1, inplace=True)


In [38]:
#listing the column names
columns=['Blue_car_counter', 'Utilib-Counter','Utilib1.4_Counter','Charge_Slots','Charge_Status','City','ID','Kind','Geo_Point','Postal_Code','Public_Name','Rental_Status','Scheduled-At','Slots','Station_Type','Status','Subscription_Status','Year','Month','Day','Hour','Minute']


In [40]:
#Preview the columns
df_autolib.columns.str.strip()


Index(['Bluecar counter', 'Utilib counter', 'Utilib 1.4 counter',
       'Charge Slots', 'Charging Status', 'City', 'ID', 'Kind', 'Postal code',
       'Public name', 'Rental status', 'Scheduled at', 'Slots', 'Station type',
       'Status', 'Subscription status', 'year', 'month', 'day', 'hour',
       'minute'],
      dtype='object')

In [42]:
#changing to lowercase
df_autolib.columns=map(str.lower, df_autolib.columns)

In [43]:
#to preview the datasetet
df_autolib.columns

Index(['bluecar counter', 'utilib counter', 'utilib 1.4 counter',
       'charge slots', 'charging status', 'city', 'id', 'kind', 'postal code',
       'public name', 'rental status', 'scheduled at', 'slots', 'station type',
       'status', 'subscription status', 'year', 'month', 'day', 'hour',
       'minute'],
      dtype='object')

Checking for missing values

In [45]:
#counting the number of missing values in our dataframe

df_autolib.count()


bluecar counter        5000
utilib counter         5000
utilib 1.4 counter     5000
charge slots           5000
charging status        5000
city                   5000
id                     5000
kind                   5000
postal code            5000
public name            5000
rental status          5000
scheduled at             47
slots                  5000
station type           5000
status                 5000
subscription status    5000
year                   5000
month                  5000
day                    5000
hour                   5000
minute                 5000
dtype: int64

In [46]:
#CHECKING FOR missing VALUES
#checking if there is any null values in our dataframe across the column

df_autolib.isnull()

,bluecar counter,utilib counter,utilib 1.4 counter,charge slots,charging status,city,id,kind,postal code,public name,rental status,scheduled at,slots,station type,status,subscription status,year,month,day,hour,minute
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
4996,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
4997,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
4998,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False


filling missing values

In [47]:
#filling missing values with fillna forward
df_fill_forward= df_autolib.fillna(method='ffill')
df_fill_forward

,bluecar counter,utilib counter,utilib 1.4 counter,charge slots,charging status,city,id,kind,postal code,public name,rental status,scheduled at,slots,station type,status,subscription status,year,month,day,hour,minute
0,0,0,0,0,nonexistent,Paris,paris-suffren-2,STATION,75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,6,0,0,0,operational,Paris,paris-raymondlosserand-145,STATION,75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
2,3,0,2,0,operational,Le Bourget,lebourget-johnfitzgeraldkennedy-2,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14
3,3,1,0,1,operational,Paris,paris-eugeneoudine-51,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37
4,3,0,0,0,nonexistent,Paris,paris-portedechamperret-6,PARKING,75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2,0,0,0,nonexistent,Colombes,colombes-menelotte-8,STATION,92700,Colombes/MÃ©nelotte/8,operational,2012-09-17T08:57:28,3,station,ok,nonexistent,2018,4,6,11,26
4996,4,0,0,1,operational,Paris,paris-dantzig-37,STATION,75015,Paris/Dantzig/37,operational,2012-09-17T08:57:28,2,station,ok,nonexistent,2018,4,4,16,56
4997,1,0,0,1,operational,Paris,paris-bac-142,STATION,75007,Paris/Bac/142,operational,2012-09-17T08:57:28,4,station,ok,nonexistent,2018,4,1,7,1
4998,2,0,0,0,nonexistent,Fontenay-Sous-Bois,fontenaysousbois-valdefontenay-2,STATION,94120,Fontenay-Sous-Bois/Val de Fontenay/2,operational,2012-09-17T08:57:28,3,station,ok,nonexistent,2018,4,4,17,27


In [48]:
#filling in missing values using fillna backward
df_fill_backward =df_fill_forward.fillna(method ='bfill')
df_fill_backward

,bluecar counter,utilib counter,utilib 1.4 counter,charge slots,charging status,city,id,kind,postal code,public name,rental status,scheduled at,slots,station type,status,subscription status,year,month,day,hour,minute
0,0,0,0,0,nonexistent,Paris,paris-suffren-2,STATION,75015,Paris/Suffren/2,operational,2016-07-27T15:32:21,2,station,ok,nonexistent,2018,4,8,11,43
1,6,0,0,0,operational,Paris,paris-raymondlosserand-145,STATION,75014,Paris/Raymond Losserand/145,operational,2016-07-27T15:32:21,0,station,ok,nonexistent,2018,4,6,7,24
2,3,0,2,0,operational,Le Bourget,lebourget-johnfitzgeraldkennedy-2,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,2016-07-27T15:32:21,1,station,ok,nonexistent,2018,4,3,20,14
3,3,1,0,1,operational,Paris,paris-eugeneoudine-51,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2016-07-27T15:32:21,2,station,ok,nonexistent,2018,4,4,4,37
4,3,0,0,0,nonexistent,Paris,paris-portedechamperret-6,PARKING,75017,Paris/Porte de Champerret/6,operational,2016-07-27T15:32:21,3,station,ok,nonexistent,2018,4,8,17,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2,0,0,0,nonexistent,Colombes,colombes-menelotte-8,STATION,92700,Colombes/MÃ©nelotte/8,operational,2012-09-17T08:57:28,3,station,ok,nonexistent,2018,4,6,11,26
4996,4,0,0,1,operational,Paris,paris-dantzig-37,STATION,75015,Paris/Dantzig/37,operational,2012-09-17T08:57:28,2,station,ok,nonexistent,2018,4,4,16,56
4997,1,0,0,1,operational,Paris,paris-bac-142,STATION,75007,Paris/Bac/142,operational,2012-09-17T08:57:28,4,station,ok,nonexistent,2018,4,1,7,1
4998,2,0,0,0,nonexistent,Fontenay-Sous-Bois,fontenaysousbois-valdefontenay-2,STATION,94120,Fontenay-Sous-Bois/Val de Fontenay/2,operational,2012-09-17T08:57:28,3,station,ok,nonexistent,2018,4,4,17,27


In [50]:
#filling invalues backward by using the new variable
df2=df_fill_backward
df2

,bluecar counter,utilib counter,utilib 1.4 counter,charge slots,charging status,city,id,kind,postal code,public name,rental status,scheduled at,slots,station type,status,subscription status,year,month,day,hour,minute
0,0,0,0,0,nonexistent,Paris,paris-suffren-2,STATION,75015,Paris/Suffren/2,operational,2016-07-27T15:32:21,2,station,ok,nonexistent,2018,4,8,11,43
1,6,0,0,0,operational,Paris,paris-raymondlosserand-145,STATION,75014,Paris/Raymond Losserand/145,operational,2016-07-27T15:32:21,0,station,ok,nonexistent,2018,4,6,7,24
2,3,0,2,0,operational,Le Bourget,lebourget-johnfitzgeraldkennedy-2,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,2016-07-27T15:32:21,1,station,ok,nonexistent,2018,4,3,20,14
3,3,1,0,1,operational,Paris,paris-eugeneoudine-51,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2016-07-27T15:32:21,2,station,ok,nonexistent,2018,4,4,4,37
4,3,0,0,0,nonexistent,Paris,paris-portedechamperret-6,PARKING,75017,Paris/Porte de Champerret/6,operational,2016-07-27T15:32:21,3,station,ok,nonexistent,2018,4,8,17,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2,0,0,0,nonexistent,Colombes,colombes-menelotte-8,STATION,92700,Colombes/MÃ©nelotte/8,operational,2012-09-17T08:57:28,3,station,ok,nonexistent,2018,4,6,11,26
4996,4,0,0,1,operational,Paris,paris-dantzig-37,STATION,75015,Paris/Dantzig/37,operational,2012-09-17T08:57:28,2,station,ok,nonexistent,2018,4,4,16,56
4997,1,0,0,1,operational,Paris,paris-bac-142,STATION,75007,Paris/Bac/142,operational,2012-09-17T08:57:28,4,station,ok,nonexistent,2018,4,1,7,1
4998,2,0,0,0,nonexistent,Fontenay-Sous-Bois,fontenaysousbois-valdefontenay-2,STATION,94120,Fontenay-Sous-Bois/Val de Fontenay/2,operational,2012-09-17T08:57:28,3,station,ok,nonexistent,2018,4,4,17,27


**ANALYSIS**

Identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018.



In [52]:
#filtering out the rows with the city of paris from the dataframe
Paris=df_autolib[df_autolib['city'].map(lambda city:'Paris' in city)]

#filtering the dataset to only have the stations that are operational

paris2=Paris[Paris['rental status']=='operational']
paris2.head(5)

,bluecar counter,utilib counter,utilib 1.4 counter,charge slots,charging status,city,id,kind,postal code,public name,rental status,scheduled at,slots,station type,status,subscription status,year,month,day,hour,minute
0,0,0,0,0,nonexistent,Paris,paris-suffren-2,STATION,75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,6,0,0,0,operational,Paris,paris-raymondlosserand-145,STATION,75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
3,3,1,0,1,operational,Paris,paris-eugeneoudine-51,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37
4,3,0,0,0,nonexistent,Paris,paris-portedechamperret-6,PARKING,75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23
5,0,0,0,0,nonexistent,Paris,paris-voltaire-8,STATION,75011,Paris/Voltaire/8,operational,NaN,4,station,ok,nonexistent,2018,4,6,7,2


In [61]:
paris2pivot=pd.pivot_table(Paris,values=['bluecar counter'],index=['hour'],aggfunc=np.sum,fill_value=0) 
paris2pivot.sort_values(by='bluecar counter',ascending=False).head(5)


,bluecar counter
hour,
21,268
6,260
12,250
9,250
3,242
